# **Cálculo de Intervalos Probables de Precios Futuros usando Volatilidad Implícita**


El código servirá para calcular intervalos probables para el precio futuro de un activo, usando su volatilidad implícita y un modelo log-normal de precios.

### 1. **Parámetros Principales**
 - $ S_0 $ : precio actual del activo
 - $\sigma $ : volatilidad anual implícita
 - $T$ : horizonte temporal en años (por ejemplo 30 días -> $\frac {30}{365}$)
 - $\mu $: drift anual (puede ser $r-q$ para risk-neutral o una expéctativa histórica para real-world).
 - $C$ : Lista de niveles de confianza (ej. 68%, 90%, 95%, 99%, ..., $n$%)

### 2. **Ajustes de volatilidad al horizonte**
La volatilidad se ajusta según el horizonte temporal usando:
$\sigma_T = \sigma \sqrt T$

### 3. **Cálculo del Z-score para cada nivel de confianza**
Para un intervalo bilateral de nivel de confianza $C$:
$$
    z= \Phi^{-1} (\frac {1+C}{2})
$$
donde $\Phi^{-1}$ es la función inversa de la normal estándar.

### 4. **Fórmulas para los níveles de intervalo**
Usando el modelo log-normal, los límites inferior y superior se calculan como: 
$$
S_low = S_0\cdot exp((\mu-0.5\sigma^{2})T - \sigma_T Z)
$$
$$
S_high = S_0\cdot exp((\mu-0.5\sigma^{2})T + \sigma_T Z)
$$ 


### **Organización de resultados**

Se creará una tabla que muestre para cada nivel de confianza:
- Límite inferior y superior.
- Porcentaje de cambio respecto a $S_0$.
- Diferenciando entre drift risk-neutral y real-world.

In [45]:
# Importar librerias
import numpy as np
import yfinance as yf
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource
from datetime import datetime

In [46]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import yfinance as yf
from datetime import datetime

def calcular_intervalos_precios(S0, sigma, T, mu_rn, mu_rw=None, conf_levels=None):
    """
    Calcula intervalos probables de precios futuros usando volatilidad implícita
    """
    if conf_levels is None:
        conf_levels = [0.25, 0.50, 0.68, 0.90, 0.95, 0.99]
    
    sigma_T = sigma * np.sqrt(T)

    drift_component_rn = (mu_rn - 0.5 * sigma**2) * T
    if mu_rw is not None:
        drift_component_rw = (mu_rw - 0.5 * sigma**2) * T
    
    resultados = []
    for conf in conf_levels:
        z = norm.ppf((1 + conf) / 2)

        # Risk-Neutral
        S_low_rn = S0 * np.exp(drift_component_rn - sigma_T * z)
        S_high_rn = S0 * np.exp(drift_component_rn + sigma_T * z)

        resultados.append({
            'Confianza': f'{conf*100:.1f}%',
            'Tipo': 'Risk-Neutral',
            'Limite_Inferior': S_low_rn,
            'Limite_Superior': S_high_rn,
            '%_Cambio_Inferior': (S_low_rn - S0) / S0 * 100,
            '%_Cambio_Superior': (S_high_rn - S0) / S0 * 100
        })

        # Real-World
        if mu_rw is not None:
            S_low_rw = S0 * np.exp(drift_component_rw - sigma_T * z)
            S_high_rw = S0 * np.exp(drift_component_rw + sigma_T * z)

            resultados.append({
                'Confianza': f'{conf*100:.1f}%',
                'Tipo': 'Real-World',
                'Limite_Inferior': S_low_rw,
                'Limite_Superior': S_high_rw,
                '%_Cambio_Inferior': (S_low_rw - S0) / S0 * 100,
                '%_Cambio_Superior': (S_high_rw - S0) / S0 * 100
            })
    return pd.DataFrame(resultados)

# ===============================
# 🚀 Uso con datos reales
# ===============================
if __name__ == "__main__":
    name = 'AMZN'
    ticker = yf.Ticker(name)
                       
    # 1. Precio spot
    S0 = ticker.history(period="max")['Close'].iloc[-1]

    # 2. Seleccionar vencimiento (ej. 30 días más cercano)
    vencimientos = ticker.options
    hoy = datetime.today()
    vencimientos_dt = [datetime.strptime(d, "%Y-%m-%d") for d in vencimientos]
    dif_dias = [(v - hoy).days for v in vencimientos_dt]
    horizonte = 30  # días
    idx = min(range(len(dif_dias)), key=lambda i: abs(dif_dias[i] - horizonte))
    vencimiento_elegido = vencimientos[idx]

    # 3. Obtener cadena de opciones
    chain = ticker.option_chain(vencimiento_elegido)
    calls = chain.calls

    # 4. Strike más cercano al spot (ATM)
    strike_atm = calls.iloc[(calls['strike'] - S0).abs().argmin()]['strike']
    opcion_atm = calls[calls['strike'] == strike_atm].iloc[0]

    # 5. Volatilidad implícita anualizada
    sigma = opcion_atm['impliedVolatility']

    # 6. Tiempo hasta vencimiento en años
    T = dif_dias[idx] / 365

    # 7. Drift risk-neutral
    r = 0.04   # tasa libre de riesgo (4%)
    q = 0.0    # dividendos (puedes mejorarlo con datos reales)
    mu_rn = r - q

    # 8. Ejecutar cálculo
    resultados_df = calcular_intervalos_precios(
        S0=S0,
        sigma=sigma,
        T=T,
        mu_rn=mu_rn,
        mu_rw=None,  # opcional
        conf_levels=[0.25, 0.50, 0.68, 0.90, 0.95, 0.99]
    )

    pd.set_option('display.float_format', '{:.2f}'.format)
    print("Intervalos Probables de Precios Futuros:", name)
    print("="*70)
    print("Spot:", S0)
    print("Vencimiento elegido:", vencimiento_elegido, f"({dif_dias[idx]} días)")
    print("Strike ATM:", strike_atm)
    print("IV (σ):", sigma)
    print(resultados_df.to_string(index=False))


Intervalos Probables de Precios Futuros: AMZN
Spot: 219.19900512695312
Vencimiento elegido: 2025-10-24 (28 días)
Strike ATM: 220.0
IV (σ): 0.3303289624023437
Confianza         Tipo  Limite_Inferior  Limite_Superior  %_Cambio_Inferior  %_Cambio_Superior
    25.0% Risk-Neutral           212.66           225.43              -2.98               2.84
    50.0% Risk-Neutral           205.85           232.89              -6.09               6.25
    68.0% Risk-Neutral           199.91           239.81              -8.80               9.40
    90.0% Risk-Neutral           188.36           254.51             -14.07              16.11
    95.0% Risk-Neutral           183.01           261.96             -16.51              19.51
    99.0% Risk-Neutral           172.98           277.14             -21.08              26.43


In [47]:
# Graficamos resultados
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource

def graficar_intervalos(df, S0):
    """
    Grafica los intervalos de confianza usando Bokeh
    """
    # Filtramos Risk-Neutral (puedes añadir Real-World si quieres compararlos)
    df_rn = df[df['Tipo'] == 'Risk-Neutral'].copy()
    
    # Fuente de datos para Bokeh
    source = ColumnDataSource(data=dict(
        confianza=df_rn['Confianza'],
        low=df_rn['Limite_Inferior'],
        high=df_rn['Limite_Superior'],
        cambio_low=df_rn['%_Cambio_Inferior'],
        cambio_high=df_rn['%_Cambio_Superior']
    ))

    # Crear figura
    p = figure(
        title="Intervalos Probables de Precios (Risk-Neutral)",
        x_range=df_rn['Confianza'],
        y_axis_label="Precio",
        width=800,
        height=400,
        toolbar_location="above"
    )

    # Dibujar intervalos como barras verticales
    p.segment(
        x0='confianza', y0='low',
        x1='confianza', y1='high',
        source=source, line_width=4, color="navy"
    )

    # Añadir puntos en los extremos
    p.circle(x='confianza', y='low', size=8, source=source, color="red", legend_label="Límite Inferior")
    p.circle(x='confianza', y='high', size=8, source=source, color="green", legend_label="Límite Superior")

    # Línea horizontal del spot
    p.line(x=df_rn['Confianza'], y=[S0]*len(df_rn), line_dash="dashed", color="black", legend_label="Spot")

    # Hover tooltips
    hover = HoverTool(tooltips=[
        ("Confianza", "@confianza"),
        ("Límite Inferior", "@low{0.00}"),
        ("Límite Superior", "@high{0.00}"),
        ("% Cambio Inferior", "@cambio_low{0.00}%"),
        ("% Cambio Superior", "@cambio_high{0.00}%")
    ])
    p.add_tools(hover)

    p.legend.location = "top_left"
    p.xaxis.axis_label = "Nivel de Confianza"
    p.yaxis.axis_label = "Precio estimado"
    
    show(p)


# 🔥 Uso después de tu DataFrame
graficar_intervalos(resultados_df, S0)
